# Zusatzkapitel - Wie baue ich mein eigenes Wikipediakorpus?

## Z.1. Kapitelübersicht

TODO


<b>Abschnittsübersicht</b>:<br>
[Z.1. Kapitelübersicht](#Z-1)<br>

## Z.2. Erstellung der JSON-Datei mit Wikipediakategorien

Schauen wir uns zunächst die JSON-Datei `testwikicatogeries.json` im Order `data` an. Sie hat folgenden Inhalt:<br>

```
{
 "Fernsehserie nach Staat": ["Kategorie:Fernsehserie (Australien)", "Kategorie:Fernsehserie (Belgien)"], 
 "Oper": ["Kategorie:Opernarie", "Kategorie:Bayreuther_Festspiele"]
}
``` 

Sie besteht aus einem Dictionary, bei dem die keys Wikipedia-Kategorien und die values Listen von untergeordneten Wikipedia-Kategorien sind. Um ein eigenes Korpus mit der gleichen Form zu bauen, muss eine solche JSON-Datei erstellt werden. Sind keine Unterkategorien erwünscht, muss als einziges Listenelement die Kategorie angegeben werden. Wichtig ist, dass vor den Kategorien in der values-Liste das Präfix "Kategorie:" steht.<br>

Doch wo können wir sehen, welche Kategorien es bei Wikipedia gibt? Die Sortierung von Artikeln nach Kategorien ist bei Wikipedia etwas versteckt. Sie findet sich unter der Spezial-Seite <a href="https://de.wikipedia.org/wiki/Spezial:Kategorien">Kategorien</a>. Dort kann man mithilfe der Suchmaske Kategorien finden. In Klammern steht immer die Anzahl der verfügbaren Artikel und/oder der Unterkategorien. Sollte ein nicht zu kleines Korpus erstellt werden, sollten Kategorien mit einer größeren Anzahl an Artikeln und/oder Unterkategorien ausgewählt werden.<br>

Geben wir einmal den Suchbegriff "Informatik" ein. Es existieren 10 Unterkategorien zu der Kategorie und 86 weitere Artikel (siehe <a href="https://de.wikipedia.org/wiki/Kategorie:Informatik">Kategorie:Informatik</a>). Manche Unterkategorien bestehen dabei aus nur wenigen Artikeln, diese lohnen sich meistens nicht für das Korpus. Ich würde in diesem Fall wahrscheinlich die etwas speziellere Kategorie <a href="https://de.wikipedia.org/wiki/Kategorie:Informatik_nach_Fachgebiet">"Informatik nach Fachgebiet"</a> wählen und ihre Unterkategorien in die values-Liste schreiben.<br>

Die Erstellung der JSON-Datei erfordert am meisten Arbeit und kann natürlich nicht nur händisch, sondern auch mit selbst geschriebenen Programmen und Parsern erstellt werden. Ich empfehle, einen Blick auf das Modul <a href="https://pypi.org/project/Wikipedia-API/">Wikipedia-API</a> zu werfen. Ich habe es genutzt, um die Wikipedia-Artikel im meinem CLI-Tool `wikiparser.py` zu parsen und zu extrahieren.

## Z.3. Nutzung von `wikiparser.py` <a class="anchor" id="Z-3"/>

Das CLI-Tool `wikiparser.py` kann wie alle anderen CLI-Tools über das Terminal geladen werden.

<img src="tutorialdata/img/wikiparser_1.png" alt="Wikiparser CLI-Tool Screenshot" align="center" width="600px;"/>

Es hat ein verpflichtendes Argument `path`. Dort muss der Pfad zu JSON-Datei angegeben werden. Optionale Argumente sind `--max_articles` und `--save_date`. `--max_articles` gibt die maximale Anzahl an Artikeln an, die geparst werden, um das Parsing zu beschleunigen und das Korpus nicht zu groß werden zu lassen. Der Standardwert ist 240. Dies sollte auch angegeben werden, wenn kein unausgeglichener Datensatz erwünscht ist (siehe Kapitel 5). Als Wert sollte dabei für einen ausgeglichenen Datensatz die niedrigste Anzahl an Artikeln einer Kategorie genommen werden. Ich würde persönlich empfehlen, den Wert etwas höher zu setzen als Artikel pro Kategorie erwünscht sind, da Artikel die kleiner als 100 Tokens vom Tool nicht geparst werden. Artikel, die mehr als 2000 Tokens haben, werden zugeschnitten. Fügt man dem CLI-Tool das Argument `--save_date` hinzu, wird das aktuelle Datum in die erstellte csv-Datei geschrieben.<br>

Funktionert alles, wird ein Fortschrittsbalken für jede Kategorie angezeigt. Die erstellte csv-Datei wird im Ordner `data/corpora` gespeichert.

<img src="tutorialdata/img/wikiparser_2.png" alt="Wikiparser CLI-Tool Screenshot 2" align="center" width="600px;"/>

<b>Besonderheiten vom `wikiparser.py`</b>:
- Folgende Sektionen von Wikipedia-Artikeln werden nicht gespeichert: 
    - "Literatur"
    - "Weblinks"
    - "Einzelnachweis" / "Einzelnachweise"
    - "Siehe auch"
- Artikel mit weniger als 100 Tokens werden beim Parsen übersprungen
- Artikel mit mehr als 2000 Tokens werden beim Parsen auf 2000 Tokens gekürzt
- Je nach Anzahl der Kategorien erhält die ausgegebene csv-Datei folgende Namen:
    - smallwikicorpus → 10 oder weniger Kategorien
    - wikicorpus → mehr als 10 und weniger als 51 Kategorien
    - bigwikicorpus → mehr als 50 Kategorien
- In `parserutils.py` befindet sich die auskommentierte Zeilen `# article_dict["summary"] = article.summary`. Wird die Kommentierung entfernt, kann auch die Zusammenfassung zu Beginn jedes Wikipedia-Artikel in der csv-Datei als einzelne Spalte gespeichert werden.

## Z.4. Nutzung von `data_postprocess.py` <a class="anchor" id="Z-4"/>